In [5]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [6]:
!pip install spacy
!pip install PyMuPDF
!pip install transformers
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
2023-07-06 07:19:53.846552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [7]:
import spacy
import fitz
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS
from spacy import displacy
from string import punctuation

In [8]:
# Load the English language model in SpaCy
nlp = spacy.load("en_core_web_lg")

# Extract the paragraph from pdf without extra spaces and new lines
pdf_file = "Sample.pdf"
doc = fitz.open(pdf_file)
paragraph = ""
for page_num, page in enumerate(doc.pages(), start=1):
  text = page.get_text()
  paragraph += text;
  tokens = paragraph.split()

max_similarity = 0
sample_query = nlp("This letter agreement (this 'Letter Agreement') is entered into in connection with")
selected_partition = ""

for i in range(0, len(tokens), 275):
  if i+300 < len(tokens):
    partition = " ".join(tokens[i : i+300])
  else:
    partition = " ".join(tokens[i : len(tokens)])


  # Process the paragraph using SpaCy
  doc = nlp(partition)
  if max_similarity < sample_query.similarity(doc):
    max_similarity = sample_query.similarity(doc)
    selected_partition = doc

  # Extract named entities from the processed document
  # named_entities = [(entity.text, entity.label_) for entity in doc.ents]

# removing stop_words from the tokens
selected_partition_without_stop_words = ""
for token in selected_partition:
  if not nlp.vocab[token.text].is_stop and token.text.lower() not in punctuation:
    selected_partition_without_stop_words += " " + token.text

selected_partition_without_stop_words = nlp(selected_partition_without_stop_words)

d = {}
named_entities = [(entity.text, entity.label_) for entity in selected_partition_without_stop_words.ents]
for text,label in named_entities:
  if d.get(label) == None:
    d[label] = [text]
  else:
    if text not in d[label]:
      d[label].append(text)

print('\n\n')
displacy.render(selected_partition_without_stop_words, style="ent", jupyter=True)
print('\n\n')
print(d)





{'ORG': ['Inveniam Private Equity Fund Demo IV', 'W1U', 'Fund Documents', 'Capital Commitment Inveniam Capital Partners', 'L.L.C. Delaware', 'Investor Agreements', 'Comparable Investors', 'Fund Inveniam Private Equity'], 'DATE': ['10022 January 1 2019', 'August 27', '2021', 'January 1 2019'], 'PERSON': ['Maria Sharapova'], 'FAC': ['Blandford Street'], 'GPE': ['London'], 'CARDINAL': ['3DN', '50,000,000', '1', '0.85', '2'], 'LAW': ['Letter Agreement', 'Limited Partnership Agreement Subscription Agreement Investor Fund', 'Subscription Agreement'], 'PRODUCT': ['Sharapova Investor']}
